<a href="https://colab.research.google.com/github/dspacks/MySchoolMenuNotebook/blob/main/school_menus_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# School Menu Calendar Generator

**Simple approach:** Just paste the URL from your browser!

## How to Use

1. **Run the first cell** to install/import libraries
2. **Copy a menu URL from your browser** and paste it in "Step 1: Enter Menu URLs"
3. **Run remaining cells** to generate calendars

### Where to Find URLs

1. Visit your school menu website (e.g., healthepro.com or myschoolmenus.com)
2. Navigate to your school
3. Click on a menu (Breakfast, Lunch, etc.)
4. Copy the URL from your browser address bar
5. Paste it below!

### Example URLs

- `https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948` (HealthePro)
- `https://www.myschoolmenus.com/organizations/2230/sites/14066/menus/65638` (MySchoolMenus)

In [ ]:
# Install required library
!pip install requests

In [ ]:
# Import required modules
import requests
import json
import re
import uuid
import logging
from datetime import datetime, time, date, timedelta
from dataclasses import dataclass, field
from enum import Enum
from pathlib import Path
from typing import List, Dict, Optional

logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

print("✓ Libraries imported")

## Helper Classes and Functions

In [ ]:
def parse_menu_url(url: str) -> Dict[str, int]:
    """
    Extract district_id, site_id, menu_id from menu URL.

    Example:
        parse_menu_url("https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948")
        # Returns: {'district_id': 2230, 'site_id': 14066, 'menu_id': 102948}
    """
    url = url.strip()
    pattern = r'/organizations/(\d+)/sites/(\d+)/menus/(\d+)'
    match = re.search(pattern, url)

    if not match:
        raise ValueError(f"Could not parse URL: {url}")

    district_id, site_id, menu_id = match.groups()
    return {
        'district_id': int(district_id),
        'site_id': int(site_id),
        'menu_id': int(menu_id)
    }

# Test the parser
test_url = "https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948"
result = parse_menu_url(test_url)
print(f"✓ URL Parser Working")
print(f"  Example: {result}")

In [ ]:
class MenuType(Enum):
    BREAKFAST = "breakfast"
    LUNCH = "lunch"
    SNACK = "snack"
    DINNER = "dinner"

@dataclass
class MenuItem:
    date: date
    main_dish: str
    categories: List[str] = field(default_factory=list)
    all_items: List[str] = field(default_factory=list)
    menu_type: MenuType = MenuType.LUNCH

@dataclass
class CalendarEvent:
    summary: str
    dtstart: datetime | date
    uid: str = field(default_factory=lambda: str(uuid.uuid4()))
    dtstamp: datetime = field(default_factory=datetime.now)
    description: str = ""
    dtend: Optional[datetime] = None
    transp: str = "OPAQUE"

    def to_ical_lines(self) -> List[str]:
        def escape_text(text: str) -> str:
            text = text.replace("\\", "\\\\")
            text = text.replace(",", "\\,")
            text = text.replace(";", "\\;")
            text = text.replace("\n", "\\n")
            text = text.replace("\r", "\\r")
            return text

        def fold_line(property_name: str, content: str, max_length: int = 75) -> List[str]:
            first_line = f"{property_name}:{content}"
            if len(first_line) <= max_length:
                return [first_line]
            lines = []
            current_line = first_line
            while len(current_line) > max_length:
                lines.append(current_line[:max_length])
                current_line = " " + current_line[max_length:]
            if current_line:
                lines.append(current_line)
            return lines

        lines = [
            "BEGIN:VEVENT",
            f"SUMMARY:{escape_text(self.summary)}",
        ]

        if isinstance(self.dtstart, datetime):
            lines.append(f"DTSTART:{self.dtstart.strftime('%Y%m%dT%H%M%S')}")
        else:
            lines.append(f"DTSTART;VALUE=DATE:{self.dtstart.strftime('%Y%m%d')}")

        if self.dtend:
            lines.append(f"DTEND:{self.dtend.strftime('%Y%m%dT%H%M%S')}")

        lines.append(f"DTSTAMP:{self.dtstamp.strftime('%Y%m%dT%H%M%SZ')}")
        lines.append(f"UID:{self.uid}")

        if self.description:
            desc_lines = fold_line("DESCRIPTION", escape_text(self.description))
            lines.extend(desc_lines)

        lines.append(f"TRANSP:{self.transp}")
        lines.append("END:VEVENT")
        return lines

print("✓ Data classes defined")

In [ ]:
class MenuClient:
    """API client that fetches menus from HealthePro or MySchoolMenus."""

    def __init__(self):
        self.base_urls = [
            "https://menus.healthepro.com/api",
            "https://www.myschoolmenus.com/api",
            "https://myschoolmenus.com/api"
        ]

    def fetch_menu(self, url: str, menu_type: MenuType = MenuType.LUNCH) -> List[MenuItem]:
        """Fetch menu items from URL."""
        ids = parse_menu_url(url)
        logger.info(f"Fetching: district {ids['district_id']}, menu {ids['menu_id']}...")

        endpoint = f"/organizations/{ids['district_id']}/menus/{ids['menu_id']}"

        # Try each base URL
        last_error = None
        for base_url in self.base_urls:
            try:
                response = requests.get(f"{base_url}{endpoint}", timeout=10)
                response.raise_for_status()
                data = response.json()
                break
            except Exception as e:
                last_error = e
                continue
        else:
            raise Exception(f"Failed to fetch menu: {last_error}")

        items = []
        for entry in data.get('data', []):
            if entry is None:
                continue
            try:
                item_date = datetime.fromisoformat(entry['day']).date()
                setting = json.loads(entry.get('setting', '{}'))
                display_items = setting.get('current_display', [])

                main_dish = None
                categories = []
                all_items = []

                for item in display_items:
                    if item is None:
                        continue
                    item_name = item.get('name', 'Unknown')
                    item_type = item.get('type', 'unknown')
                    all_items.append(item_name)
                    if item_type == 'recipe' and main_dish is None:
                        main_dish = item_name
                    elif item_type == 'category':
                        categories.append(item_name)

                if main_dish:
                    items.append(MenuItem(
                        date=item_date,
                        main_dish=main_dish,
                        categories=categories,
                        all_items=all_items,
                        menu_type=menu_type
                    ))
            except:
                continue

        logger.info(f"✓ Retrieved {len(items)} menu items")
        return items

print("✓ MenuClient class defined")

In [ ]:
class CalendarGenerator:
    """Generate RFC 5545-compliant iCalendar files."""

    def __init__(self, title: str = "School Menu Calendar"):
        self.title = title
        self.events = []

    def add_menu_item(self, item: MenuItem, event_time: Optional[time] = None) -> None:
        """Add menu item as calendar event."""
        prefix = item.menu_type.value[0].upper()
        summary = f"{prefix}: {item.main_dish}"

        description_parts = []
        for dish in item.all_items:
            if dish == dish.upper() or dish.endswith(':'):
                if description_parts:
                    description_parts.append("")
                description_parts.append(dish)
            else:
                description_parts.append(dish)

        description = "\\n".join(description_parts)

        if event_time:
            dtstart = datetime.combine(item.date, event_time)
            duration = timedelta(minutes=30 if item.menu_type == MenuType.BREAKFAST else 45)
            dtend = dtstart + duration
        else:
            dtstart = item.date
            dtend = None

        event = CalendarEvent(
            summary=summary,
            dtstart=dtstart,
            dtend=dtend,
            description=description
        )
        self.events.append(event)

    def add_menu_items(self, items: List[MenuItem], event_time: Optional[time] = None) -> None:
        """Add multiple menu items."""
        for item in items:
            self.add_menu_item(item, event_time)

    def to_ics(self) -> str:
        """Generate iCalendar string."""
        lines = [
            "BEGIN:VCALENDAR",
            "VERSION:2.0",
            "PRODID:-//MenuAPI//EN",
            f"X-WR-CALNAME:{self.title}",
            "CALSCALE:GREGORIAN",
            "METHOD:PUBLISH"
        ]

        for event in self.events:
            lines.extend(event.to_ical_lines())

        lines.append("END:VCALENDAR")
        return "\r\n".join(lines)

    def save(self, filepath: str) -> None:
        """Save to .ics file."""
        Path(filepath).parent.mkdir(parents=True, exist_ok=True)
        with open(filepath, 'w', newline='') as f:
            f.write(self.to_ics())
        print(f"✓ Saved calendar to {filepath}")

print("✓ CalendarGenerator class defined")

## Step 1: Enter Menu URLs

Copy and paste your menu URLs below. You can have multiple URLs (breakfast, lunch, different schools, etc.)

In [ ]:
# PASTE YOUR MENU URLS HERE
menu_urls = [
    # Example: "https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948",
    # Add your URLs below:
]

if not menu_urls:
    print("⚠️  No URLs provided. Add menu URLs above and run again.")
else:
    print(f"✓ {len(menu_urls)} URL(s) configured")
    for i, url in enumerate(menu_urls, 1):
        try:
            ids = parse_menu_url(url)
            print(f"  {i}. District {ids['district_id']}, Menu {ids['menu_id']} ✓")
        except ValueError as e:
            print(f"  {i}. ERROR: {e}")

## Step 2: Configure Menu Types and Times

For each URL, specify whether it's breakfast or lunch (determines event time in calendar).

In [ ]:
# CONFIGURE MENU TYPES
# Format: (url, menu_type, event_time)
# menu_type: MenuType.BREAKFAST or MenuType.LUNCH
# event_time: time(hour, minute) or None for all-day events

menu_configs = [
    # ("https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948", MenuType.BREAKFAST, time(8, 0)),
    # ("https://menus.healthepro.com/organizations/2230/sites/14066/menus/102946", MenuType.LUNCH, time(12, 0)),
]

if not menu_configs:
    print("⚠️  No menu configurations. Add configs above and run again.")
else:
    print(f"✓ {len(menu_configs)} menu configuration(s)")
    for i, (url, mtype, etime) in enumerate(menu_configs, 1):
        print(f"  {i}. {mtype.value.upper()} @ {etime}")

## Step 3: Generate Calendars

In [ ]:
client = MenuClient()
calendar = CalendarGenerator("School Menu Calendar")

for url, menu_type, event_time in menu_configs:
    try:
        items = client.fetch_menu(url, menu_type)
        calendar.add_menu_items(items, event_time=event_time)
    except Exception as e:
        print(f"✗ Error processing {url}: {e}")

if calendar.events:
    print(f"\n✓ Total calendar events: {len(calendar.events)}")
else:
    print("⚠️  No calendar events generated")

## Step 4: Save and Download

In [ ]:
if calendar.events:
    output_file = "school_menu_calendar.ics"
    calendar.save(output_file)
    print(f"\n✓ Calendar saved: {output_file}")
    print(f"\nYou can now:")
    print(f"1. Download the .ics file from the left sidebar")
    print(f"2. Import into Google Calendar, Outlook, Apple Calendar, or any calendar app")
    print(f"3. Or open in your calendar application directly")
else:
    print("⚠️  No calendar to save. Check steps above for errors.")

## Optional: View Calendar Data

In [ ]:
import pandas as pd

if calendar.events:
    # Extract menu items from calendar
    data = []
    for event in calendar.events:
        data.append({
            'date': event.dtstart if isinstance(event.dtstart, date) else event.dtstart.date(),
            'menu': event.summary,
            'description_preview': event.description[:50] + '...' if len(event.description) > 50 else event.description
        })

    df = pd.DataFrame(data)
    print("Menu Calendar Preview:")
    print(df.to_string(index=False))

    # Save as CSV too
    df.to_csv('menu_data.csv', index=False)
    print("\n✓ Also saved menu data to menu_data.csv")
else:
    print("No calendar events to display")